# Fun with Neural Nets

---

Below is a procedure for building a neural network to recognize handwritten digits.  The data is from Kaggle, and you will submit your results to Kaggle to test how well you did!

1. Load the training data (`train.csv`) from Kaggle
2. Setup X and y (feature matrix and target vector)
3. Split X and y into train and test subsets.
4. Preprocess your data
   - When dealing with image data, you need to normalize your `X` by dividing each value by the max value of a pixel (255).
   - Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix
   
5. Create your network.
   - Remember that for multi-class classification you need a softmax activation function on the output layer.
   - You may want to consider using regularization or dropout to improve performance.
   
6. Train your network.
7. If you are unhappy with your model performance, try to tighten up your model by adding hidden layers, adding hidden layer units, chaining the activation functions on the hidden layers, etc.
8. Load in Kaggle's `test.csv`
9. Create your predictions (these should be numbers in the range 0-9).
10. Save your predictions and submit them to Kaggle.

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

np.random.seed(1016)

# Reading in the image data
digits = pd.read_csv('../../Datasets/train.csv')

Using TensorFlow backend.


# Keras Sequential

In [3]:
# Let's check for a target variable
digits.head(1)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# Good first step here
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [31]:
X_train = X_train.values
X_test  = X_test.values

In [39]:
# Pre-processing our target variable as well, making them categorical for digits 0-9
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [46]:
model = Sequential()
n_input = X_train.shape[1]
n_hidden = n_input
n_output = Y_train.shape[1]


model.add(Dense(units = n_hidden,
                input_dim=n_input,
                activation='relu'))
model.add(Dense(units = n_output, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [48]:
model.fit(X_train, Y_train, 
                 validation_data=(X_test, Y_test),
                 epochs=400,
                 verbose=0)

In [52]:
KerasSqTest = pd.read_csv('../../Datasets/NNtest.csv')

In [55]:
KerasSqTest.shape

(28000, 784)

In [56]:
predictions = model.predict_classes(KerasSqTest, verbose=0)

submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("NNtest_1.csv", index=False, header=True)

This model did not go well. (Kaggle score = 0.108)

# Convolutional Neural Network

In [ ]:
# Reading it in again
digits = pd.read_csv('../../Datasets/train.csv')

In [3]:
# Let's check for the target variable again because it's a good habit
digits.head(1)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# We'll be using label as the target variable, we want our network to correctly identify hand-written
# with the pixel information
X = digits.drop('label', axis=1)
y = digits.label

In [6]:
# As per usual
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [6]:
# Pre-processing our data to make sure we can work with our pixel data
X_train = X_train.values.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.values.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [7]:
# Pre-processing our target variable as well, making them categorical for digits 0-9
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [16]:
# Setting up model architecture. Starting with Sequential so our poor model doesn't get lost along the way
model = Sequential()

# Creating the first model; let's make sure it works before we go crazy
model.add(Convolution2D(filters = 6, 
                        kernel_size = 3,
                       activation ='relu',
                       input_shape = (28,28,1)))

In [17]:
# Let's go to the pool
model.add(MaxPooling2D(pool_size= (2,2)))

In [18]:
model.add(Convolution2D(filters = 16,
                       kernel_size = 3,
                       activation ='relu'))

model.add(MaxPooling2D(pool_size= (2,2)))

model.add(Convolution2D(filters = 8,
                       kernel_size = 3,
                       activation ='relu'))

model.add(MaxPooling2D(pool_size= (2,2)))

#Let's throw a dropout in
model.add(Dropout(0.25))

In [19]:
# Need to make this 2 dimensional
model.add(Flatten())

In [20]:
# We're back to having a bunch of 2 dimensional nodes, we're continuing the neural network treatment here
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

In [21]:
# 6. Compiling model, using categorical cross entropy because we're using softmax for our output
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [13]:
# 7. Fitting model on training data, fingers crossed
model.fit(X_train, Y_train, 
          batch_size=32, epochs=10, verbose=1)

Epoch 1/10
29400/29400 [==============================] - 8s 269us/step - loss: 1.2054 - acc: 0.5753
Epoch 2/10
29400/29400 [==============================] - 8s 284us/step - loss: 0.7667 - acc: 0.7379
Epoch 3/10
29400/29400 [==============================] - 8s 275us/step - loss: 0.6551 - acc: 0.7812
Epoch 4/10
29400/29400 [==============================] - 8s 271us/step - loss: 0.5980 - acc: 0.8015
Epoch 5/10
29400/29400 [==============================] - 8s 278us/step - loss: 0.5642 - acc: 0.8157
Epoch 6/10
29400/29400 [==============================] - 8s 265us/step - loss: 0.5236 - acc: 0.8270
Epoch 7/10
29400/29400 [==============================] - 7s 255us/step - loss: 0.5000 - acc: 0.8355
Epoch 8/10
29400/29400 [==============================] - 7s 248us/step - loss: 0.4808 - acc: 0.8438
Epoch 9/10
29400/29400 [==============================] - 7s 249us/step - loss: 0.4590 - acc: 0.8494
Epoch 10/10
29400/29400 [==============================] - 7s 238us/step - loss: 0.4393 - a

In [14]:
# 8. Evaluate model on test data
score = model.evaluate(X_test, Y_test, verbose=0)
labels = model.metrics_names

In [15]:
# We did it bois(?)
print(str(labels[0]) + ": " + str(score[0]))
print(str(labels[1]) + ": " + str(score[1]))

loss: 0.24535427847551922
acc: 0.9350793651172093


### Test Data

In [16]:
# Test data

digitz = pd.read_csv('../../Datasets/NNtest_2.csv')

In [17]:
digitz.shape

(28000, 784)

In [18]:
digitz.head(1)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
X = digitz

In [22]:
X.shape

(28000, 28, 28, 1)

In [21]:
X = X.values.reshape(X.shape[0], 28, 28, 1)
X = X.astype('float32')
X /= 255

In [23]:
predictions = model.predict_classes(X, verbose=0)

submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("NNtest_2.csv", index=False, header=True)

Kaggle score = 0.927